### Text Classification With Machine Learning and SpaCy
+ Text categorization / text classification is the task of assigning predefined categories to documents.
+ Sentiment Analysis
+ Multilabel classification
+ + DataSet source http://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences

##### Aim is to classify reviews into positive or negative review


In [764]:
# Load EDA packages
import pandas as pd

In [765]:
# Load our dataset
df_yelp = pd.read_table('yelp_labelled.txt')
df_imdb = pd.read_table('imdb_labelled.txt')
df_amz = pd.read_table('amazon_cells_labelled.txt')

In [766]:
# Concatenate our Datasets
frames = [df_yelp,df_imdb,df_amz]

In [767]:
# Renaming Column Headers
for colname in frames:
    colname.columns = ["Message","Target"]

In [768]:
# Column names
for colname in frames:
    print(colname.columns)

Index(['Message', 'Target'], dtype='object')
Index(['Message', 'Target'], dtype='object')
Index(['Message', 'Target'], dtype='object')


In [769]:
# Assign a Key to Make it Easier
keys = ['Yelp','IMDB','Amazon']

In [770]:
# Merge or Concat our Datasets
df = pd.concat(frames,keys=keys)

In [771]:
# Length and Shape 
df.shape

(2745, 2)

In [772]:
df.head()

Message  Target
Yelp 0                                 Crust is not good.       0
     1          Not tasty and the texture was just nasty.       0
     2  Stopped by during the late May bank holiday of...       1
     3  The selection on the menu was great and so wer...       1
     4     Now I am getting angry and I want my damn pho.       0

In [773]:
df.to_csv("sentimentdataset.csv")

In [774]:
# Data Cleaning
df.columns

Index(['Message', 'Target'], dtype='object')

In [775]:
# Checking for Missing Values
df.isnull().sum()

Message    0
Target     0
dtype: int64

###  Working with SpaCy
+ Removing Stopwords
+ Lemmatizing

In [776]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en_core_web_trf')

In [777]:
# Build a list of stopwords to use to filter
stopwords = list(STOP_WORDS)

In [778]:
stopwords

['whereas',
 'most',
 'even',
 'among',
 'eleven',
 'get',
 'further',
 'beyond',
 'whoever',
 'may',
 'no',
 'side',
 'due',
 'across',
 'up',
 'why',
 'down',
 'nor',
 'already',
 'name',
 'will',
 'themselves',
 'of',
 'also',
 'seems',
 'bottom',
 'three',
 'hundred',
 'what',
 'myself',
 'beside',
 'together',
 'about',
 'latter',
 'mostly',
 'anyhow',
 'more',
 'after',
 'except',
 'part',
 'while',
 'make',
 'or',
 'should',
 'indeed',
 'whereby',
 'mine',
 'formerly',
 'two',
 'above',
 'around',
 'nothing',
 '’s',
 'toward',
 'whenever',
 'perhaps',
 'moreover',
 'have',
 '‘ll',
 'since',
 'seemed',
 'see',
 'with',
 'amount',
 'give',
 'me',
 'call',
 'used',
 'hereby',
 'last',
 'towards',
 'though',
 'sixty',
 'top',
 'per',
 'never',
 'move',
 'than',
 'almost',
 'made',
 'and',
 'did',
 'whole',
 'else',
 'our',
 'he',
 'anything',
 'for',
 'back',
 'although',
 'only',
 'meanwhile',
 'say',
 'am',
 'some',
 'might',
 'between',
 'twenty',
 'quite',
 'using',
 'hers',
 'a

##### Getting Lemma and Stop words

In [779]:
docx = nlp("This is how John Walker was walking. He was also running beside the lawn.")

In [780]:
# Lemmatizing of tokens
for word in docx:
    print(word.text,"Lemma =>",word.lemma_)

This Lemma => this
is Lemma => be
how Lemma => how
John Lemma => John
Walker Lemma => Walker
was Lemma => be
walking Lemma => walk
. Lemma => .
He Lemma => he
was Lemma => be
also Lemma => also
running Lemma => run
beside Lemma => beside
the Lemma => the
lawn Lemma => lawn
. Lemma => .


In [781]:
# Lemma that are not pronouns
for word in docx:
    if word.lemma_ != "-PRON-":
        print(word.lemma_.lower().strip())

this
be
how
john
walker
be
walk
.
he
be
also
run
beside
the
lawn
.


In [782]:
# List Comprehensions of our Lemma
[word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in docx]

['this',
 'be',
 'how',
 'john',
 'walker',
 'be',
 'walk',
 '.',
 'he',
 'be',
 'also',
 'run',
 'beside',
 'the',
 'lawn',
 '.']

In [783]:
# Filtering out Stopwords and Punctuations
for word in docx:
    if word.is_stop == False and not word.is_punct:
#     if word.is_stop != True and not word.is_punct:
        print(word)

John
Walker
walking
running
lawn


In [784]:
# Stop words and Punctuation In List Comprehension
[ word for word in docx if word.is_stop == False and not word.is_punct ]

[John, Walker, walking, running, lawn]

In [785]:
# Use the punctuations of string module
import string
punctuations = string.punctuation

In [786]:
# Creating a Spacy Parser
from spacy.lang.en import English
parser = English()

In [787]:
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    #mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

#### Machine Learning With SKlearn

In [788]:
# ML Packages
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [789]:
#Custom transformer using spaCy 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Basic function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [790]:
# Vectorization
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
classifier = LinearSVC(dual=False)

In [791]:
# Using Tfidf
tfvectorizer = TfidfVectorizer()

In [792]:
# Splitting Data Set
from sklearn.model_selection import train_test_split

In [793]:
# Features and Labels
X = df['Message']
ylabels = df['Target']

In [794]:
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.1, random_state=42)

In [795]:
# Create the  pipeline to clean, tokenize, vectorize, and classify 
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

In [796]:
X_train.to_csv("test.csv")

In [797]:
# Fit our data
pipe.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000001DD959E6F10>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x000001DE0B80BEE0>)),
                ('classifier', LinearSVC(dual=False))])

In [798]:
# Predicting with a test dataset
sample_prediction = pipe.predict(X_test)

In [799]:
# Prediction Results
# 1 = Positive review
# 0 = Negative review
for (sample,pred) in zip(X_test,sample_prediction):
    print(sample,"Prediction=>",pred)

Great pork sandwich. Prediction=> 1
It is a true classic.   Prediction=> 1
It's close to my house, it's low-key, non-fancy, affordable prices, good food. Prediction=> 1
Audio Quality is poor, very poor. Prediction=> 1
We loved the biscuits!!! Prediction=> 1
I don't have very many words to say about this place, but it does everything pretty well. Prediction=> 1
Was not happy. Prediction=> 1
The headsets are easy to use and everyone loves them. Prediction=> 1
I miss it and wish they had one in Philadelphia! Prediction=> 1
Still it's quite interesting and entertaining to follow.   Prediction=> 1
All three broke within two months of use. Prediction=> 1
Oh yeah, and the storyline was pathetic too.   Prediction=> 1
IT'S REALLY EASY. Prediction=> 1
Every element of this story was so over the top, excessively phony and contrived that it was painful to sit through.   Prediction=> 1
The food was outstanding and the prices were very reasonable. Prediction=> 1
I am so tired of clichés that is just

In [800]:
# Accuracy
print("Accuracy: ",pipe.score(X_test,y_test))
print("Accuracy: ",pipe.score(X_test,sample_prediction))

Accuracy:  0.49818181818181817
Accuracy:  1.0


In [801]:
# Accuracy
print("Accuracy: ",pipe.score(X_train,y_train))

Accuracy:  0.4951417004048583


In [802]:
# Another random review
pipe.predict(["This was a great movie"])

array([1], dtype=int64)

In [803]:
example = ["I do enjoy my job",
 "What a poor product!,I will have to get a new one",
 "I feel amazing!"]

In [804]:
pipe.predict(example)

array([1, 1, 1], dtype=int64)

In [805]:
#### Using Tfid

In [806]:
# Create the  pipeline to clean, tokenize, vectorize, and classify 
pipe_tfid = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfvectorizer),
                 ('classifier', classifier)])

In [807]:
pipe_tfid.fit(X_train,y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x000001DD959F28E0>),
                ('vectorizer', TfidfVectorizer()),
                ('classifier', LinearSVC(dual=False))])

In [808]:
sample_prediction1 = pipe_tfid.predict(X_test)

In [809]:
for (sample,pred) in zip(X_test,sample_prediction1):
    print(sample,"Prediction=>", pred)

Great pork sandwich. Prediction=> 1
It is a true classic.   Prediction=> 1
It's close to my house, it's low-key, non-fancy, affordable prices, good food. Prediction=> 0
Audio Quality is poor, very poor. Prediction=> 0
We loved the biscuits!!! Prediction=> 1
I don't have very many words to say about this place, but it does everything pretty well. Prediction=> 1
Was not happy. Prediction=> 0
The headsets are easy to use and everyone loves them. Prediction=> 1
I miss it and wish they had one in Philadelphia! Prediction=> 1
Still it's quite interesting and entertaining to follow.   Prediction=> 1
All three broke within two months of use. Prediction=> 0
Oh yeah, and the storyline was pathetic too.   Prediction=> 0
IT'S REALLY EASY. Prediction=> 1
Every element of this story was so over the top, excessively phony and contrived that it was painful to sit through.   Prediction=> 1
The food was outstanding and the prices were very reasonable. Prediction=> 1
I am so tired of clichés that is just

In [810]:
print("Accuracy: ",pipe_tfid.score(X_test,y_test))
print("Accuracy: ",pipe_tfid.score(X_test,sample_prediction1))

Accuracy:  0.8072727272727273
Accuracy:  1.0


In [811]:
### Jesse JCharis
### J-Secur1ty
### Jesus Saves @ JCharisTech